# Notebook 03: Modeling & Evaluation

Notebook này tập trung vào việc **xây dựng, huấn luyện và đánh giá mô hình Collaborative Filtering** cho hệ thống gợi ý sản phẩm từ dữ liệu Amazon Beauty Ratings.

Trong các notebook trước, quá trình Data Exploration đã chỉ ra rằng:

- Ma trận user–item **rất thưa** (sparsity cực cao).  
- Phần lớn người dùng có **rất ít đánh giá** => Cold Start problem nghiêm trọng.  
- Phần lớn sản phẩm cũng chỉ có 1–2 đánh giá => Item Cold Start cũng tồn tại.
- Điều này khiến các mô hình Memory-based CF như **User-based / Item-based CF** hoạt động kém hiệu quả vì:  
  - Không có đủ neighbors tương đồng.  
  - Similarity-based methods dễ nhiễu khi dữ liệu quá thưa.

**Matrix Factorization giúp giảm chiều và “học” latent factors**, khắc phục hiệu quả sparsity. Do đó, mô hình Matrix Factorization là lựa chọn tốt cho hệ thống gợi ý này.

## Step 1: Import Libraries & Setup

In [2]:
import sys
sys.path.append('../src')

from pathlib import Path
import numpy as np

from models import (
    load_raw_csv_ratings,
    load_user_feature_dict,
    load_item_feature_dict,
    filter_top_users,
    group_shuffle_split,
    PureNumpyMF,
    PureNumpyMFConfig,
    regression_metrics,
)

print(' NumPy Matrix Factorization pipeline ready!')


 NumPy Matrix Factorization pipeline ready!


## Step 2: Load Data

In [3]:
RAW_CSV = Path('../data/raw/ratings_Beauty.csv')
user_ids, item_ids, ratings = load_raw_csv_ratings(str(RAW_CSV))

print(f'Ratings: {len(ratings):,}')
print(f'Unique users : {np.unique(user_ids).size:,}')
print(f'Unique items : {np.unique(item_ids).size:,}')
print(f'Mean rating  : {ratings.mean():.4f}')


Ratings: 2,023,070
Unique users : 1,210,271
Unique items : 249,274
Mean rating  : 4.1490


## Step 3: Train-Test Split

In [5]:
train_idx, test_idx = group_shuffle_split(user_ids, test_ratio=0.2, random_state=42)

train_users = user_ids[train_idx]
train_items = item_ids[train_idx]
train_ratings = ratings[train_idx]

test_users = user_ids[test_idx]
test_items = item_ids[test_idx]
test_ratings = ratings[test_idx]

print(f'Train size: {train_users.size:,} ratings')
print(f'Test size : {test_users.size:,} ratings')


Train size: 1,620,121 ratings
Test size : 402,949 ratings


## Step 4: Model Training
- Model: PureNumpyMF (chi tiết trong models.py)
- Config: use_user_features=True, use_item_features=True - tích hợp feature engineering từ Notebook 01
- SGD-based Matrix Factorization: sử dụng Stochastic Gradient Descent để tối ưu hóa hàm mất mát, implement bằng numpy thuần, có regularization để giảm overfitting.

In [6]:
config = PureNumpyMFConfig(use_user_features=True, use_item_features=True)  
np_mf = PureNumpyMF(config)

train_metrics = np_mf.fit(
    train_users,
    train_items,
    train_ratings,
)
print(f' Training RMSE: {train_metrics["RMSE"]:.4f}, MAE: {train_metrics["MAE"]:.4f}')


 Training RMSE: 0.9761, MAE: 0.5663


In [8]:
print('TEST METRICS:')
test_preds = np_mf.predict(test_users, test_items)
test_metrics = regression_metrics(test_ratings, test_preds)
print(f' Test RMSE: {test_metrics["RMSE"]:.4f}, MAE: {test_metrics["MAE"]:.4f}')

TEST METRICS:
 Test RMSE: 1.3142, MAE: 1.0520


In [9]:
sample_user = train_users[0]
rated_mask = train_users == sample_user
rated_items = set(train_items[rated_mask])

recommendations = np_mf.recommend(sample_user, n=10, exclude_items=rated_items)

print(f'Sample user: {sample_user}')
print('Already rated:', len(rated_items))
print('Top 10 recommendations:')
for row in recommendations:
    print(f"  {row['product_id']}: {row['predicted_rating']:.3f}")


Sample user: A39HTATAQ9V7YF
Already rated: 4
Top 10 recommendations:
  B0048O2R1E: 5.000
  B0002JKPA4: 5.000
  B0083QNBCM: 5.000
  B006K9OQSC: 5.000
  B0068Y6CA4: 5.000
  B0009V1YR8: 5.000
  B001JKTTVQ: 5.000
  B007BLN17K: 5.000
  B00E68O4JU: 5.000
  B009GYVMAS: 5.000


### Phân tích và đánh giá kết quả

| Metric | Training | Test |
|--------|----------|------|
| **RMSE** | **0.9761** | **1.3142** |
| **MAE**  | **0.5663** | **1.0520** |

#### **1. Sai số tăng khá nhiều từ Training -> Test**
- RMSE tăng từ **0.97 -> 1.31**  
- MAE tăng từ **0.56 -> 1.05**

Điều này cho thấy mô hình **học tốt trên dữ liệu đã thấy**, nhưng **khả năng tổng quát hoá trên dữ liệu mới còn hạn chế**.

Đây là hiện tượng phổ biến trong **Matrix Factorization** khi:

- Số lượng rating mỗi user ít => latent vectors không ổn định  
- Ma trận đánh giá quá thưa => khó học được latent factors chính xác  
- Dữ liệu test chứa nhiều trường hợp gần như "cold start"

#### **2. Training RMSE < 1 là khá tốt**
Với thang điểm rating 1-5:

- RMSE = 0.97 nói rằng dự đoán trung bình chỉ lệch xấp xỉ 1 đơn vị  
- MAE = 0.56 cho thấy dự đoán trung bình sai khoảng 0.5 sao

=> Mô hình **nắm bắt được cấu trúc nội tại của dữ liệu**.

#### **3. MAE giảm mạnh cho thấy phân phối rating lệch về high values**
MAE tăng mạnh từ **0.56 đến 1.05** gợi ý:

- Nhiều rating trong test rơi vào các item mà user ít thông tin  
- MF không thể dự đoán chính xác trong các scenario sparse hoặc cold start  
- Test set có thể chứa các sản phẩm với rất ít rating lịch sử

### Kết luận:
- *Overfitting nhẹ + ảnh hưởng cold start* là nguyên nhân chính làm test error cao.  
- Tuy vậy, mức sai số test hiện tại vẫn **nằm trong khoảng hợp lý** đối với bài toán Amazon MF.  
- Mô hình đã hoạt động **ổn** cho một Matrix Factorization thuần NumPy.
